# Membuat Dataset yang diperlukan

Pada notebook ini, kalian akan diarahkan bagaimana membuat atau menambahkan dataset SIBI language sesuai dengan keinginan kalian. Adapun library yang diperlukan pada notebook ini antara lain

<ul>
    <li>pandas</li>
    <li>numpy</li>
    <li>opencv</li>
    <li>mediapipe</li>
    <li>csv</li>
    <li>pickle</li>
</ul>

In [77]:
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
import pickle
import csv

## Menggunakan Library Mediapipe untuk Pose Estimation 

Pada project ini, digunakan library mediapipe yang dapat membantu kita dalam melakukan pose estimation, yaitu sebuah pekerjaan pada CV untuk menentukan posisi setiap titik-titik penting pada anggota tubuh. Pada sesi ini kita gunakan pose estimation untuk jari-jari kita

In [78]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [65]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
    
cap.release()
cv2.destroyAllWindows()

## Pengambilan Dataset

Dataset pada project ini berupa video rekaman kalian yang menunjukkan beberapa posisi jari yang melambangkan beberapa kata atau huruf pada SIBI Language. Adapun SIBI language adalah salah satu bahasa isyarat yang dipergunakan di Indonesia. Adapun yang perlu kalian perhatikan dalam pengambilan dataset
 
 1. Tentukan dahulu, huruf atau kata apa yang akan kalian peragakan. 
 2. Usahakan posisi jari-jari kalian terlihat jelas pada webcam kalian. 
 3. lama durasi pada setiap recording label dapat disesuaikan kebutuhan (untuk dataset yang sudah ada dilakukan rekaman kurang lebih 40 detik pada setiap label)

In [99]:
label_data = 'Halo'

In [100]:
# Define global variable for hand keypoints data
hand_keypoint_data = np.array([])

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    # Read until video is completed
  while cap.isOpened():
    # Capture frame-by-frame
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    
    try:
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
          mp_drawing.draw_landmarks(
              image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())
      
      # Checking keypoints if complete will do this block of code
      if len(results.multi_hand_landmarks[0].landmark) >= 21:
        # define variable for centering and scaling process
        centering = np.array([])
        scaling = np.array([])

        # Centering X coordinate Process
        for indexPoint in range(21):
          centering = np.append(centering, (
            results.multi_hand_landmarks[0].landmark[0].x - results.multi_hand_landmarks[0].landmark[indexPoint].x))

        # Centering Y coordinate Process
        for indexPoint in range(21):
          centering = np.append(centering, (
            results.multi_hand_landmarks[0].landmark[0].y - results.multi_hand_landmarks[0].landmark[indexPoint].y))

        centering = centering.reshape(2, 21)
        
        # Scaling Process
        for indexIter in range(2):
          for jointIter in range(21):
            scaling = np.append(scaling, centering[indexIter][jointIter] / np.max(
              np.absolute(centering[indexIter])) * 320)
        
        # Normalization Process
        for jointIter in range(42):
          hand_keypoint_data = np.append(hand_keypoint_data, (scaling[jointIter] + 320))

        # Write spatiodata from hand keypoints coordinate
        if len(hand_keypoint_data) >= 210:
          # Write spatiodata to csv
          with open('./data/data_'+ label_data + '.csv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(hand_keypoint_data)

          # deleted 42 old data 
          deletedIndex = np.arange(42)
          hand_keypoint_data = np.delete(hand_keypoint_data, deletedIndex)

    except Exception as e:
      continue

    finally:
      # Show the result
      cv2.imshow('Result', image)

      if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()